In [8]:
import numpy as np
import pandas as pd
from copy import deepcopy
import matplotlib.pyplot as plt

In [9]:
##Setup
dayInYear = 1/365
zeroPercent = 20 # under 20
standardLoan = 6.9

In [10]:
#all interest assumed in nexo for easy payback
#value in usd
class Asset():
    balance = 0
    collatoral = 0
    
    def __init__(self, name, percentInterestInNexo, maxLTV, balance, valueOfSingle, special=False, collatoral=0):
        self.name = name
        self.percentInterestInNexo = percentInterestInNexo
        self.maxLTV = maxLTV
        self.balance = balance
        self.valueOfSingle = valueOfSingle
        self.interestSpecial = special
        self.value = self.balance * self.valueOfSingle
        self.collatoral = collatoral # in the asset numbers
        self.collatoralValue = self.collatoral * self.valueOfSingle
        self.interestPerDay = self.calcInterestPerDay()
        
    def __str__(self):
        return "name: {} percent {} balance {} Collatoral {}".format(
            self.name, self.percentInterestInNexo, self.balance, self.collatoral)
    
    def __repr__(self):
        return self.__str__()

    def calcInterestPerDay(self):
        return self.calcInterestPerYear() * dayInYear
    
    def calcInterestPerYear(self):
        collatoral = self.collatoralValue
        if (self.interestSpecial):
            collatoral = 0
        self.interestPerYear = ((self.value - collatoral) * self.percentInterestInNexo) / 100.0
        return self.interestPerYear

In [11]:
def plotGraphltvInterestLoanAmountCustom(customPortfolio, maxLoan):
#     %matplotlib inline 
    #widget
    nexo = np.arange(0,maxLoan, 100) #15000
    allResults = [testScenarioCustom(val, deepcopy(customPortfolio)) for val in nexo]
    y = [result[0] for result in allResults]
    
    color = 'tab:blue'
    fig, ax1 = plt.subplots()
    ax1.set_ylabel('Interest per day(USD)', color=color)
#     plt.ylabel("Interest per day(USD)")
    plt.xlabel("Loan(USD)")
#     plt.xticks(np.arange(0,15000, 100))
    axes2 = plt.twinx()
    y2 = [result[1] for result in allResults]
    
    color = 'tab:red'
    axes2.set_ylabel("LTV", color=color)
    axes2.plot(nexo,y2, color=color, alpha=0.5, label="LTV")
    axes2.tick_params(axis='y', label1On=False)
    ax1.plot(nexo,y, label="Interest per day(USD)")
    color = 'tab:purple'
    axes2.plot(nexo,[15 for x in nexo], color=color, label="NEXO LTV LIMIT", alpha=0.7) # Change to just represent nexo LTV
    #TODO add LTV lines for other assets, ideally a single combined one
    plt.legend()
#     ax1.yaxis.set_visible(False) # Hide only x axis
#     ax1.xaxis.set_visible(False) # Hide only x axis
    plt.show()

In [15]:
#get LTV of existing loan
def getLTVofLoan(assetsList):
    return (sum([x.balance for x in assetsList if x.name == "LOANUSDC"])/sum([asset.collatoralValue for asset in assetsList]))*100.0

def getCollatoralOfLoan(assetsList):
    return sum([asset.collatoralValue for asset in assetsList])

def getInterestPerDayIncludingColatoral(assetsList):
    return sum([asset.interestPerDay for asset in assetsList])

def getInterestPerYearIncludingColatoral(assetsList):
    return sum([asset.interestPerYear for asset in assetsList])

def calculateAPR(assetsList):
    return (getInterestPerYearIncludingColatoral(assetsList)/getTotalValue(assetsList)) * 100.0

def getTotalValue(assetsList):
    return sum([asset.value for asset in assetsList])

In [12]:
def testScenarioCustom(amount, custom, printdebug=False):
    customPortfolio = deepcopy(custom)
    if(printdebug):
        print("LOAN AMOUNT : ${}".format(amount))
    custom.append(Asset("LOANUSDC", standardLoan, 0, amount, -1))
    custom.append(Asset("LOANEDUSDC", 12, 90.0, amount, 1))
    
    LTV = getLTVofLoan(custom)
    if(printdebug):
        print(LTV)
    loanPercent = standardLoan
    if (LTV < zeroPercent):
        loanPercent = 0.0
    customPortfolio.append(Asset("LOANUSDC", loanPercent, 0, amount, -1))
    customPortfolio.append(Asset("LOANEDUSDC", 12, 90.0, amount, 1))

    LTV = getLTVofLoan(customPortfolio)
    if(printdebug):
        print(LTV)
    if(printdebug):
        getDetailsOfAssets(customPortfolio)
    return (getInterestPerDayIncludingColatoral(customPortfolio), LTV, getInterestPerYearIncludingColatoral(customPortfolio), calculateAPR(customPortfolio), customPortfolio)

In [14]:
from collections import defaultdict
import csv
from decimal import Decimal as D
##Analyse code

In [15]:
def readInPortfolioFromCsv(path, ignoreInOut, ignoreOut):
    types = set()
    wallet = defaultdict(D)
    detailedWallet = defaultdict(lambda: defaultdict(D))
    ignoredinout = defaultdict(lambda: defaultdict(D))

    loanSet = False
    loan = 0

    with open(path,"r") as csvfile:
        reader = csv.DictReader(csvfile)
        i = 0
        for l in reader:
            inCur = l['Input Currency']
            outCur = l['Output Currency']
            types.add(l['Type'])
            if l['Type'] in ignoreInOut:
                ignoredinout[l['Type']][inCur] += D(l['Input Amount'])
                ignoredinout[l['Type']+"o"][outCur] += D(l['Output Amount'])
                continue

            wallet[inCur] += D(l['Input Amount'])
            detailedWallet[l['Type']][inCur] += D(l['Input Amount'])
            if l['Type'] not in ignoreOut:
                wallet[outCur] += D(l['Output Amount'])
                detailedWallet[l['Type']+"o"][outCur] += D(l['Output Amount'])    

            i+=1

            #get outstanding loan
            if i> 1 and not loanSet and D(l['Outstanding Loan'][1:]) != 0:
                loan = D(l['Outstanding Loan'][1:])
                loanSet = True

    #usd and GBP are not included as made into X straight away
    if 'USD' in wallet.keys():
        del wallet['USD']
    if 'GBP' in wallet.keys():
        del wallet['GBP']
    if 'EUR' in wallet.keys():
        del wallet['EUR']
    if '' in wallet.keys():
        del wallet['']

    #trim any that are presently 0
    toDel = []
    for key in wallet.keys():
        if wallet[key] == 0:
            toDel.append(key)

    for key in toDel:
        del wallet[key]
    return wallet,detailedWallet,loan,types